# Write cache for future hydro

In [1]:
from loca import print_date
print_date()

Last executed: 2019-08-07 15:33:19.086524 by jvano on casper11


# Imports and Options

In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import xarray as xr

from loca.data_catalog import load_monthly_cmip_hydro_datasets, resample_monthly_data
from loca.utils import calc_change


#cache is written in script "write_cache.ipynb"  TODO
read_cache = True
# cache_dir = os.environ['TMPDIR']
cache_dir = '/glade/u/home/jvano/scratch/'



# TODO still need to clean this up, possibly add others - or migrate this script to another place.  
# LIke that have ability to do individual models

# CACHED:
# models = ['ACCESS1-0', 'CanESM2']
# models = ['MIROC5']
# models = ['MRI-CGCM3']
# models = ['bcc-csm1-1']
# models = ['bcc-csm1-1-m']
# models = ['CCSM4']
# models = ['CESM1-BGC']
# models = ['CMCC-CM']
# models = ['GISS-E2-R']
# models = ['HadGEM2-CC']
# models = ['HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR']
# models = ['MPI-ESM-MR']
# models = ['GFDL-ESM2G'] 
# models = ['NorESM1-M']
# models = ['MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4']
# models = ['CESM1-BGC', 'CESM1-CAM5', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC','HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M']
# models = ['GFDL-ESM2G']
# models = ['CNRM-CM5']
# models = ['CSIRO-Mk3-6-0']
# models = ['GFDL-ESM2M'] 
# models = ['GFDL-CM3'] 
# models = ['HadGEM2-AO']

# list of 25, without issues:
models = ['ACCESS1-0', 'CanESM2', 'MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4',
          'CESM1-BGC', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR',
          'MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G', 
          'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

models = ['ACCESS1-0', 'CanESM2']

In [3]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.join(os.environ['HOME'], 'scheduler_file.json'))
client

Client Scheduler: tcp://128.117.181.217:42734 Dashboard: proxy/46697/status,Cluster Workers: 2 Cores: 4 Memory: 8.00 GB


# Load Data

In [4]:
write_cache = True  

if write_cache:
    xr.set_options(file_cache_maxsize=12000)
    hist_data = load_monthly_cmip_hydro_datasets('historical', models=models, autoclose=False, parallel=True)
    rcp4_data = load_monthly_cmip_hydro_datasets('rcp45', models=models, autoclose=False, parallel=True)
    rcp8_data = load_monthly_cmip_hydro_datasets('rcp85', models=models, autoclose=False, parallel=True)

In [5]:
if write_cache:
    for key, ds in hist_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_hist.{key}_{gcm}.nc'))
    for key, ds in rcp4_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_rcp4.{key}_{gcm}.nc'))
    for key, ds in rcp8_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_hydro_rcp8.{key}_{gcm}.nc'))

In [6]:
#test to see if reads in

read_cache = True
hist_data = {}
rcp4_data = {}
rcp8_data = {}
if read_cache:
    for key in ['loca', 'bcsd']:
        hist_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_hist.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp4_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_rcp4.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp8_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_hydro_rcp8.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})

OSError: no files to open

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/distributed/utils.py", line 649, in log_errors
    yield
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/distributed/client.py", line 873, in _reconnect
    yield self._ensure_connected()
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/tornado/gen.py", line 1147, in run
    yielded = self.gen.send(value)
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/distributed/client.py", line 904, in _ensure_connected
    assert len(msg) == 1
AssertionError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/glade/u/home/jvano/miniconda/envs/loca/lib/python3.6/site-packages/distributed/utils.py", line 649, in log_errors
 